In [12]:
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import os
import time
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
import torch
from torch import nn, optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.class_weight import compute_sample_weight
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import LabelEncoder
import warnings
import shutil
import tqdm
warnings.filterwarnings("ignore")

In [ ]:
#check if cuda is available to use
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device, "is used")

In [ ]:
#Process image data for feature extraction using CNN
input_dir = '../CS610_AML_Group_Project/resized_images'
full_set = datasets.ImageFolder(input_dir)

In [ ]:
#get classes from directory
num_classes = len(full_set.classes)
class_names = full_set.classes
print("Number of classes (Full Set):", num_classes,"\n", full_set.classes)

In [45]:
best_model = torch.load("/content/CS610_AML_Group_Project/model_bank/best_cnn_resnet50.pth")
cnn_model = models.resnet50(pretrained=True)
cnn_model.fc = nn.Linear(cnn_model.fc.in_features, len(class_names))
cnn_model = cnn_model.to(device)
cnn_model.load_state_dict(best_model)

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees = 15),
    transforms.ColorJitter(brightness=0.1, contrast = 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],std=[0.229,0.224,0.225]) #ImageNet
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
     transforms.Normalize([0.485,0.456,0.406],std=[0.229,0.224,0.225]) #ImageNet
])

train_size = int(0.7*len(full_set))
val_size = int(0.2*len(full_set))
test_size = len(full_set)-train_size-val_size
split_datasets = random_split(
    full_set,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

train_indices = split_datasets[0].indices
val_indices = split_datasets[1].indices
test_indices = split_datasets[2].indices

class CustomSubsetWithTransform(torch.utils.data.Dataset):
    def __init__(self, dataset, indices, transform=None):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform

    def __getitem__(self, idx):
        original_idx = self.indices[idx]
        img, label = self.dataset[original_idx]
        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.indices)
train_img_dataset = CustomSubsetWithTransform(full_set, train_indices, train_transform)
val_img_dataset = CustomSubsetWithTransform(full_set, val_indices, val_test_transform)
test_img_dataset = CustomSubsetWithTransform(full_set, test_indices, val_test_transform)

batch_size = 32
train_loader = DataLoader(train_img_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_img_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_img_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)
print("Data processing done")

Data processing done


In [5]:
#get classes from directory
num_classes = len(full_set.classes)
class_names = full_set.classes
print("Number of classes (Full Set):", num_classes,"\n", full_set.classes)

Number of classes (Full Set): 50 
 ['adidas_forum_high', 'adidas_forum_low', 'adidas_gazelle', 'adidas_nmd_r1', 'adidas_samba', 'adidas_stan_smith', 'adidas_superstar', 'adidas_ultraboost', 'asics_gel-lyte_iii', 'converse_chuck_70_high', 'converse_chuck_70_low', 'converse_chuck_taylor_all-star_high', 'converse_chuck_taylor_all-star_low', 'converse_one_star', 'new_balance_327', 'new_balance_550', 'new_balance_574', 'new_balance_990', 'new_balance_992', 'nike_air_force_1_high', 'nike_air_force_1_low', 'nike_air_force_1_mid', 'nike_air_jordan_11', 'nike_air_jordan_1_high', 'nike_air_jordan_1_low', 'nike_air_jordan_3', 'nike_air_jordan_4', 'nike_air_max_1', 'nike_air_max_270', 'nike_air_max_90', 'nike_air_max_95', 'nike_air_max_97', 'nike_air_max_plus_(tn)', 'nike_air_vapormax_flyknit', 'nike_air_vapormax_plus', 'nike_blazer_mid_77', 'nike_cortez', 'nike_dunk_high', 'nike_dunk_low', 'puma_suede_classic', 'reebok_classic_leather', 'reebok_club_c_85', 'salomon_xt-6', 'vans_authentic', 'vans_

In [15]:
#load best cnn_model
best_model = torch.load("/content/CS610_AML_Group_Project/model_bank/best_cnn_resnet50.pth")
cnn_model = models.resnet50(pretrained=True)
cnn_model.fc = nn.Linear(cnn_model.fc.in_features, len(class_names))
cnn_model = cnn_model.to(device)
cnn_model.load_state_dict(best_model)

<All keys matched successfully>

In [32]:
#create dictionary to store metrics
train_holder = {}
train_holder['y_true'], train_holder['y_hat'] = [], []
#start evaluation of model
cnn_model.eval()
train_corrects = 0
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        output = cnn_model(images)
        _, train_preds = torch.max(output, 1)
        train_corrects += (train_preds == labels).sum().item()
        train_holder['y_true'].extend(list(labels.cpu().detach().numpy()))
        train_holder['y_hat'].extend(list(train_preds.cpu().detach().numpy()))


train_y_true_all = train_holder['y_true']
train_y_pred_all = train_holder['y_hat']
train_acc = train_corrects / len(train_loader.dataset)
train_precision = precision_score(train_y_true_all, train_y_pred_all, average='macro')
train_recall = recall_score(train_y_true_all, train_y_pred_all, average='macro')
train_fbeta = fbeta_score(train_y_true_all, train_y_pred_all, beta=0.5,average='macro')
train_class_report = classification_report(train_y_true_all, train_y_pred_all)

In [33]:
#create dictionary to store metrics
test_holder = {}
test_holder['y_true'], test_holder['y_hat'] = [], []
#start evaluation of model
cnn_model.eval()
test_corrects = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        output = cnn_model(images)
        _, test_preds = torch.max(output, 1)
        test_corrects += (test_preds == labels).sum().item()
        test_holder['y_true'].extend(list(labels.cpu().detach().numpy()))
        test_holder['y_hat'].extend(list(test_preds.cpu().detach().numpy()))


test_y_true_all = test_holder['y_true']
test_y_pred_all = test_holder['y_hat']
test_acc = test_corrects / len(test_loader.dataset)
test_precision = precision_score(test_y_true_all, test_y_pred_all, average='macro')
test_recall = recall_score(test_y_true_all, test_y_pred_all, average='macro')
test_fbeta = fbeta_score(test_y_true_all, test_y_pred_all, beta=0.5,average='macro')
test_class_report = classification_report(test_y_true_all, test_y_pred_all)

In [34]:
#create dictionary to store metrics
val_holder = {}
val_holder['y_true'], val_holder['y_hat'] = [], []
#start evaluation of model
cnn_model.eval()
val_corrects = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        output = cnn_model(images)
        _, val_preds = torch.max(output, 1)
        val_corrects += (val_preds == labels).sum().item()
        val_holder['y_true'].extend(list(labels.cpu().detach().numpy()))
        val_holder['y_hat'].extend(list(val_preds.cpu().detach().numpy()))


val_y_true_all = val_holder['y_true']
val_y_pred_all = val_holder['y_hat']
val_acc = val_corrects / len(val_loader.dataset)
val_precision = precision_score(val_y_true_all, val_y_pred_all, average='macro')
val_recall = recall_score(val_y_true_all, val_y_pred_all, average='macro')
val_fbeta = fbeta_score(val_y_true_all, val_y_pred_all, beta=0.5,average='macro')
val_class_report = classification_report(val_y_true_all, val_y_pred_all)

In [35]:
print("="*5 + 'Training Metrics'+'='*5)
print(f"Accuracy: {train_acc:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F0.5-Score: {train_fbeta:.4f}")

print("="*5 + 'Validation Metrics'+'='*2)
print(f"Accuracy: {val_acc:.4f}")
print(f"Precision: {val_precision:.4f}")
print(f"Recall: {val_recall:.4f}")
print(f"F0.5-Score: {val_fbeta:.4f}")

print("="*5 + 'Testing Metrics'+'='*5)
print(f"Accuracy: {test_acc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F0.5-Score: {test_fbeta:.4f}")

=====Training Metrics=====
Accuracy: 0.9843
Precision: 0.9843
Recall: 0.9842
F0.5-Score: 0.9841
=====Validation Metrics==
Accuracy: 0.8194
Precision: 0.8335
Recall: 0.8183
F0.5-Score: 0.8247
=====Testing Metrics=====
Accuracy: 0.7901
Precision: 0.8087
Recall: 0.7859
F0.5-Score: 0.7969
